In [1]:
#import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
# Feature Scaling
from sklearn.preprocessing import StandardScaler
# NLTK libraries
import nltk
# cleaning the text
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#Creating the Bag of words model
from sklearn.feature_extraction.text import CountVectorizer
# Fitting Naive Bayes to the Training set
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import classification_report
from sklearn import metrics

#Classification library
from sklearn.tree import DecisionTreeClassifier

#Classification library
from sklearn.ensemble import RandomForestClassifier

# precision recall
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from inspect import signature
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt
%matplotlib inline

# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import importlib.util

##### A chat bot typically has 3 things in it
    Intent ( Intention of the query asked by the user)
    
    Entities ( Named entities in Query like , Location names, People names, date and etc…)  #NamedEntityRecognization
    
    Action or Response ( the result to throw back to the user)

Ex: what’s the weather in Seattle tomorrow??
    
    Here Intent → Weather check

    Entities → Seattle ( Location), tomorrow (Date)

    Response → “The weather in {Location} {Date} is so and so”

In [2]:
#Load dataset
dataset1 = pd.read_csv ("chatbot_faq.csv")
dataset = dataset1
dataset1.head()

,Question,DomainIntent,Intent,Answer,Class
0,How long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,How much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,What is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,Can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,How do I get in?,fellowship program,enter,You will require to register for one of our r...,general


In [4]:
import pandas_profiling as pda

In [5]:
df_new = pd.read_csv ("chatbot_faq.csv")


In [6]:
df_new.profile_report()

In [3]:
#Load dataset
dataset2 = pd.read_csv ('Greetings.csv')
dataset2.head()

,Question,DomainIntent,Intent,Answer,Class
0,Hello,fellowship program,greetings,Hi,greetings
1,Hi,fellowship program,greetings,Hello,greetings
2,Hii,fellowship program,greetings,Hello,greetings
3,Hey,fellowship program,greetings,Hi,greetings
4,Hey There,fellowship program,greetings,Hey,greetings


In [4]:
print("Dataset1 has {} rows and {} Columns".format(dataset1.shape[0],dataset1.shape[1])) 
print("Dataset2 has {} rows and {} Columns".format(dataset2.shape[0],dataset2.shape[1])) 

Dataset1 has 20 rows and 5 Columns
Dataset2 has 21 rows and 5 Columns


In [5]:
# check dataset information
dataset1.info(), print('\t\t'),dataset2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
Question        20 non-null object
DomainIntent    20 non-null object
Intent          20 non-null object
Answer          20 non-null object
Class           20 non-null object
dtypes: object(5)
memory usage: 880.0+ bytes
		
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 5 columns):
Question        21 non-null object
DomainIntent    21 non-null object
Intent          21 non-null object
Answer          21 non-null object
Class           21 non-null object
dtypes: object(5)
memory usage: 920.0+ bytes


(None, None, None)

In [6]:
# descibe the dataset
dataset1.describe() 

,Question,DomainIntent,Intent,Answer,Class
count,20,20,20,20,20
unique,20,1,19,17,1
top,Do fellowship program offer any stipend or liv...,fellowship program,admission_fees,Our fellows get salaries benchmarked with the...,general
freq,1,20,2,2,20


In [7]:
dataset2.describe()

,Question,DomainIntent,Intent,Answer,Class
count,21,21,21,21,21
unique,21,1,1,17,1
top,Good Evening,fellowship program,greetings,Hey,greetings
freq,1,21,21,2,21


In [8]:
# handling missing data if nessesary
dataset2.isnull().sum()

Question        0
DomainIntent    0
Intent          0
Answer          0
Class           0
dtype: int64

In [9]:
# check for duplicate values
dataset1.duplicated().sum(), dataset2.duplicated().sum()

(0, 0)

In [10]:
# Download nltk stopwords
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/admin1/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
dataset1.columns

Index(['Question', 'DomainIntent', 'Intent', 'Answer', 'Class'], dtype='object')

In [12]:
dataset2.columns

Index(['Question', 'DomainIntent', 'Intent', 'Answer', 'Class'], dtype='object')

In [13]:
# Merging both the data sets as tweets in both the data set is unstructured
combine_df = dataset1.append(dataset2, ignore_index = True, sort = False)
combine_df.head(25)

,Question,DomainIntent,Intent,Answer,Class
0,How long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,How much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,What is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,Can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,How do I get in?,fellowship program,enter,You will require to register for one of our r...,general
5,How do I register for the fellowship program?,fellowship program,register,You can register here: Link,general
6,Do fellowship program offer any stipend or liv...,fellowship program,stipend accomodations,We are currently helping deserving students w...,general
7,What do I need to know to be able to get throu...,fellowship program,coding,Try our sample MCQ questions from the Coding ...,general
8,What is the minimum qualification for the fell...,fellowship program,qualification,All the candidates need to be Engineers or MC...,general
9,What can I expect in the technical interview ?,fellowship program,interview selection,The first interview depending on your skills ...,general


In [14]:
combine_df.shape

(41, 5)

In [15]:
combine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 5 columns):
Question        41 non-null object
DomainIntent    41 non-null object
Intent          41 non-null object
Answer          41 non-null object
Class           41 non-null object
dtypes: object(5)
memory usage: 1.7+ KB


In [16]:
combine_df['Question'] = combine_df['Question'].apply(lambda x: x.lower())
combine_df.head()

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in?,fellowship program,enter,You will require to register for one of our r...,general


In [17]:
# Apostrophe Dictionary
apostrophe_dict = {
"ain't": "am not / are not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is",
"i'd": "I had / I would",
"i'd've": "I would have",
"i'll": "I shall / I will",
"i'll've": "I shall have / I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}
apostrophe_dict

{"ain't": 'am not / are not',
 "aren't": 'are not / am not',
 "can't": 'cannot',
 "can't've": 'cannot have',
 "'cause": 'because',
 "could've": 'could have',
 "couldn't": 'could not',
 "couldn't've": 'could not have',
 "didn't": 'did not',
 "doesn't": 'does not',
 "don't": 'do not',
 "hadn't": 'had not',
 "hadn't've": 'had not have',
 "hasn't": 'has not',
 "haven't": 'have not',
 "he'd": 'he had / he would',
 "he'd've": 'he would have',
 "he'll": 'he shall / he will',
 "he'll've": 'he shall have / he will have',
 "he's": 'he has / he is',
 "how'd": 'how did',
 "how'd'y": 'how do you',
 "how'll": 'how will',
 "how's": 'how has / how is',
 "i'd": 'I had / I would',
 "i'd've": 'I would have',
 "i'll": 'I shall / I will',
 "i'll've": 'I shall have / I will have',
 "i'm": 'I am',
 "i've": 'I have',
 "isn't": 'is not',
 "it'd": 'it had / it would',
 "it'd've": 'it would have',
 "it'll": 'it shall / it will',
 "it'll've": 'it shall have / it will have',
 "it's": 'it has / it is',
 "let's": 'l

In [18]:
def lookup_dict(text, dictionary):
    # 1st we have to split the sentences
    for word in text.split():
        if word.lower() in dictionary:
            if word.lower() in text.split():
                text = text.replace(word, dictionary[word.lower()])
    return text

In [19]:
combine_df['Question'] = combine_df['Question'].apply(lambda x: lookup_dict(x,apostrophe_dict))
combine_df.head(10)

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in?,fellowship program,enter,You will require to register for one of our r...,general
5,how do i register for the fellowship program?,fellowship program,register,You can register here: Link,general
6,do fellowship program offer any stipend or liv...,fellowship program,stipend accomodations,We are currently helping deserving students w...,general
7,what do i need to know to be able to get throu...,fellowship program,coding,Try our sample MCQ questions from the Coding ...,general
8,what is the minimum qualification for the fell...,fellowship program,qualification,All the candidates need to be Engineers or MC...,general
9,what can i expect in the technical interview ?,fellowship program,interview selection,The first interview depending on your skills ...,general


In [20]:
short_word_dict = {
"121": "one to one",
"a/s/l": "age, sex, location",
"adn": "any day now",
"afaik": "as far as I know",
"afk": "away from keyboard",
"aight": "alright",
"alol": "actually laughing out loud",
"b4": "before",
"b4n": "bye for now",
"bak": "back at the keyboard",
"bf": "boyfriend",
"bff": "best friends forever",
"bfn": "bye for now",
"bg": "big grin",
"bta": "but then again",
"btw": "by the way",
"cid": "crying in disgrace",
"cnp": "continued in my next post",
"cp": "chat post",
"cu": "see you",
"cul": "see you later",
"cul8r": "see you later",
"cya": "bye",
"cyo": "see you online",
"dbau": "doing business as usual",
"fud": "fear, uncertainty, and doubt",
"fwiw": "for what it's worth",
"fyi": "for your information",
"g": "grin",
"g2g": "got to go",
"ga": "go ahead",
"gal": "get a life",
"gf": "girlfriend",
"gfn": "gone for now",
"gmbo": "giggling my butt off",
"gmta": "great minds think alike",
"h8": "hate",
"hagn": "have a good night",
"hdop": "help delete online predators",
"hhis": "hanging head in shame",
"iac": "in any case",
"ianal": "I am not a lawyer",
"ic": "I see",
"idk": "I don't know",
"imao": "in my arrogant opinion",
"imnsho": "in my not so humble opinion",
"imo": "in my opinion",
"iow": "in other words",
"ipn": "I’m posting naked",
"irl": "in real life",
"jk": "just kidding",
"l8r": "later",
"ld": "later, dude",
"ldr": "long distance relationship",
"llta": "lots and lots of thunderous applause",
"lmao": "laugh my ass off",
"lmirl": "let's meet in real life",
"lol": "laugh out loud",
"ltr": "longterm relationship",
"lulab": "love you like a brother",
"lulas": "love you like a sister",
"luv": "love",
"m/f": "male or female",
"m8": "mate",
"milf": "mother I would like to fuck",
"oll": "online love",
"omg": "oh my god",
"otoh": "on the other hand",
"pir": "parent in room",
"ppl": "people",
"r": "are",
"rofl": "roll on the floor laughing",
"rpg": "role playing games",
"ru": "are you",
"shid": "slaps head in disgust",
"somy": "sick of me yet",
"sot": "short of time",
"thanx": "thanks",
"thx": "thanks",
"ttyl": "talk to you later",
"u": "you",
"ur": "you are",
"uw": "you’re welcome",
"wb": "welcome back",
"wfm": "works for me",
"wibni": "wouldn't it be nice if",
"wtf": "what the fuck",
"wtg": "way to go",
"wtgp": "want to go private",
"ym": "young man",
"gr8": "great"
}

In [21]:
combine_df['Question'] = combine_df['Question'].apply(lambda x: lookup_dict(x,short_word_dict))
combine_df.head()

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in?,fellowship program,enter,You will require to register for one of our r...,general


##### Replacing Special Characters, Numbers (integers)  with space

In [22]:
combine_df['Question'] = combine_df['Question'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))
combine_df.head()

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much does fellowship program cost,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remotely,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in,fellowship program,enter,You will require to register for one of our r...,general


##### Removing words whom length is 1

In [23]:
# combine_df['Question'] = combine_df['Question'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>1]))
# combine_df['Question'][0:5]

##### Spelling Correction - With TextBlob Library

In [24]:
combine_df['Question'] = combine_df['Question'].apply(lambda x: re.sub(r'[^\w\s]',' ',x))
combine_df.head(22)

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much does fellowship program cost,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remotely,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in,fellowship program,enter,You will require to register for one of our r...,general
5,how do i register for the fellowship program,fellowship program,register,You can register here: Link,general
6,do fellowship program offer any stipend or liv...,fellowship program,stipend accomodations,We are currently helping deserving students w...,general
7,what do i need to know to be able to get throu...,fellowship program,coding,Try our sample MCQ questions from the Coding ...,general
8,what is the minimum qualification for the fell...,fellowship program,qualification,All the candidates need to be Engineers or MC...,general
9,what can i expect in the technical interview,fellowship program,interview selection,The first interview depending on your skills ...,general


##### Replacing Special Characters, Numbers (integers)  with space

In [25]:
# Speling correlation with textblob
from textblob import TextBlob
# Not cleaning the just showing the spelling check as its take lot of time to process all these tweets
## Shown sample how its must done
text = combine_df['Question'][0:10].apply(lambda x: str(TextBlob(x).correct()))
text

0                  how long is the fellowship program 
1               how much does fellowship program cost 
2                     what is the fellowship program  
3           can the fellowship program be done remote 
4                                     how do i get in 
5        how do i register for the fellowship program 
6    do fellowship program offer any spend or livin...
7    what do i need to know to be able to get throu...
8    what is the minimum qualification for the fell...
9       what can i expect in the technical interview  
Name: Question, dtype: object

### 1.word_tokenize

In [26]:
# Importing stop words from NLTK coupus and word tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

In [27]:
# # raw=combine_df.lower()# converts to lowercase
# nltk.download('punkt') # first-time use only (puctuation)
# nltk.download('wordnet') # first-time use only
# sent_tokens = nltk.sent_tokenize(combine_df['Question'] )# converts to list of sentences 
# word_tokens = nltk.word_tokenize(combine_df['Question'] )# converts to list of words

In [28]:
# Creating token for the clean tweets
combine_df['Question'] = combine_df['Question'].apply(lambda x: word_tokenize(x))

## Fully formated tweets & there tokens
combine_df.head()

,Question,DomainIntent,Intent,Answer,Class
0,"[how, long, is, the, fellowship, program]",fellowship program,duration,The program is 4 months on a full-time basis.,general
1,"[how, much, does, fellowship, program, cost]",fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,"[what, is, the, fellowship, program]",fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,"[can, the, fellowship, program, be, done, remo...",fellowship program,remote work,No! We believe that interaction with the ment...,general
4,"[how, do, i, get, in]",fellowship program,enter,You will require to register for one of our r...,general


### 2.stopwords

Do not use stopwords, bcoz it remove the questions also, like how what etc

In [29]:
# Importing stop words from NLTK corpus for english language
stop_words = set(stopwords.words('english'))
# stop_words

In [30]:
# # Created new columns of tokens - where stop words are being removed
# combine_df['Question_filtered'] = combine_df['Question'].apply(lambda x: [word for word in x if not word in stop_words])

# ## Tokens columns with stop words and without stop words
# combine_df[['Question', 'Question_filtered']].head()

### 3.Stemming & Lemmatization

In [31]:
#It take rootwords using stem() func by importing PorterStemmer
stemming = PorterStemmer()

In [32]:
# Created one more columns tweet_stemmed it shows tweets' stemmed version
combine_df['Question'] = combine_df['Question'].apply(lambda x: ' '.join([stemming.stem(i) for i in x]))
combine_df['Question'].head(10)

0                   how long is the fellowship program
1                 how much doe fellowship program cost
2                       what is the fellowship program
3             can the fellowship program be done remot
4                                      how do i get in
5           how do i regist for the fellowship program
6    do fellowship program offer ani stipend or liv...
7    what do i need to know to be abl to get throug...
8    what is the minimum qualif for the fellowship ...
9           what can i expect in the technic interview
Name: Question, dtype: object

### 4 . Lemmatization

In [33]:
# Importing library for stemming
from nltk.stem import PorterStemmer
# Importing library for lemmatizing
from nltk.stem.wordnet import WordNetLemmatizer
#visualizing all the words in column "tweet_stemmed" in our data using the wordcloud plot.
from wordcloud import WordCloud
# Importing library
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
lemmatizing = WordNetLemmatizer()
nltk.download('punkt') # first-time use only (puctuation)
nltk.download('wordnet') # first-time use only

[nltk_data] Downloading package punkt to /home/admin1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/admin1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
# sent_tokens = nltk.sent_tokenize(combine_df['Question'])# converts to list of sentences 
# word_tokens = nltk.word_tokenize(combine_df['Question'])# converts to list of words
# combine_df['Question'] = combine_df['Question'].apply(lambda x: word_tokenize(x))

In [36]:
import string # to process standard python strings

# LemTokens will take as input the tokens and return normalized tokens.
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)

def LemNormalize(text):
#     combine_df['Question'] = combine_df['Question'].apply(lambda x: word_tokenize(x))
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [37]:
# # raw=combine_df.lower()# converts to lowercase
# nltk.download('punkt') # first-time use only (puctuation)
# nltk.download('wordnet') # first-time use only
# sent_tokens = nltk.sent_tokenize(combine_df['Question'] )# converts to list of sentences 
# word_tokens = nltk.word_tokenize(combine_df['Question'] )# converts to list of words

In [38]:
# # Created one more columns Question it shows tweets' lemmatized version
# combine_df['Question'] = combine_df['Question'].apply(lambda x: ' '.join([lemmatizing.lemmatize(i) for i in x]))
# combine_df['Question'].head()

In [39]:
combine_df.head(25)

,Question,DomainIntent,Intent,Answer,Class
0,how long is the fellowship program,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,how much doe fellowship program cost,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,what is the fellowship program,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,can the fellowship program be done remot,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,how do i get in,fellowship program,enter,You will require to register for one of our r...,general
5,how do i regist for the fellowship program,fellowship program,register,You can register here: Link,general
6,do fellowship program offer ani stipend or liv...,fellowship program,stipend accomodations,We are currently helping deserving students w...,general
7,what do i need to know to be abl to get throug...,fellowship program,coding,Try our sample MCQ questions from the Coding ...,general
8,what is the minimum qualif for the fellowship ...,fellowship program,qualification,All the candidates need to be Engineers or MC...,general
9,what can i expect in the technic interview,fellowship program,interview selection,The first interview depending on your skills ...,general


In [40]:
import nltk
import random
from sklearn.feature_extraction.text import TfidfVectorizer
# Also, import cosine similarity module from scikit learn library
from sklearn.metrics.pairwise import cosine_similarity

In [41]:
# convert whole dataframe as a string to perform operation on it
raw = combine_df.to_string()
print("...", type(raw))

sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

... <class 'str'>


In [43]:

# # handle greeting response
# def greeting(sentence):
#     for word in sentence.split():
#         if  word in combine_df['Class'] == 'greetings':
#             if word.lower() in combine_df['Question']:
#                 return random.choice(combine_df['Answer'])
#         else:
#             print(response(user_response))
            
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
# handle greeting response
def greeting(sentence):
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
                return random.choice(GREETING_RESPONSES)
        
        
def response(user_response):
    robo_response=''
#     # sent_tokens = sent_tokenize(combine_df['Question'] )
#     sent_tokens = combine_df['Question'].apply(lambda x: word_tokenize(x)).tolist()
    
    sent_tokens.append(user_response)
#     print(".........",type(sent_tokens))
    
    TfidfVec = TfidfVectorizer(tokenizer = LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    
#     print(".........",type(sent_tokens))
    
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    
    flat.sort()
    req_tfidf = flat[-2]
    
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

flag=True
print("ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!")
while(flag==True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            flag=False
            print("ROBO: You are welcome..")
        else:
            if(greeting(user_response)!= None):
                print("ROBO: "+greeting(user_response))
            else:
                print("ROBO: ",end="")
                print("...............",response(user_response))
                sent_tokens.remove(user_response)
    else:
        flag=False
        print("ROBO: Bye! take care..")

ROBO: My name is Robo. I will answer your queries about Chatbots. If you want to exit, type Bye!
hi
ROBO: hello
bye
ROBO: Bye! take care..


In [44]:
dataset.head()

,Question,DomainIntent,Intent,Answer,Class
0,How long is the fellowship program?,fellowship program,duration,The program is 4 months on a full-time basis.,general
1,How much does fellowship program cost?,fellowship program,admission_fees,The program is free to the fellows. You do no...,general
2,What is the fellowship program ?,fellowship program,fellowship program,Coding jobs with emerging tech product compani...,general
3,Can the fellowship program be done remotely?,fellowship program,remote work,No! We believe that interaction with the ment...,general
4,How do I get in?,fellowship program,enter,You will require to register for one of our r...,general


In [45]:
# def greeting(sentence):
 
#     for word in sentence.split():
#         if word.lower() in GREETING_INPUTS:
#             return random.choice(GREETING_RESPONSES)

In [46]:
# cleaning the text
corpus = []
for i in range(dataset.shape[0]):
    # step1 -> Only taking letters
    # Here we only taking latter, others are removed
    ques = re.sub('[^a-zA-Z]', ' ', dataset['Question'][i])
#     print(review)

    # step2 -> Convert all words into lower case
    ques = ques.lower()
#     print(review)
    # split to array(default delimiter is " ") 
    # Step3 -> take sentence into words
    #  Tokenization, involves splitting sentences and words from the body of the text.
    ques = ques.split()
#     print(review)

    # Step4 -> Remove Unnecessary words using stopwords 
    # Step5 -> Take rootwords using stem() func by importing PorterStemmer

    ps = PorterStemmer()
    # here we take set() function bcoz it goes through all the word faster to all words in list
    ques = [ps.stem(word) for word in ques if not word in set(stopwords.words('english'))]
#     print(review)

    # Step6 -> join all words
    ques = ' '.join(ques)
    
    # Step7 -> Append whole list
    corpus.append(ques)
    


In [47]:
ques

'need pay fellowship program'

In [48]:
cv = CountVectorizer(max_features=1500)
x = cv.fit_transform(corpus).toarray()
y = dataset.iloc[:, 1].values

In [49]:
x.shape, y.shape

((20, 49), (20,))

In [50]:
import importlib.util
from sklearn.model_selection import train_test_split

In [51]:
# importing template file 
spec = importlib.util.spec_from_file_location("Util_class", "/home/admin1/PycharmProjects/Machine-Learning/Week 13/Util/util.py")
foo = importlib.util.module_from_spec(spec)
spec.loader.exec_module(foo)
# creating object of Template class
obj_util = foo.Util_class()

In [53]:
# import os
# os.mkdir('CSV_files')

In [54]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = 0.20, random_state=0)
print("x_train : ", x_train.shape, " x_test : ", x_test.shape)

df1 = pd.DataFrame(x_test)
df2 = pd.DataFrame(y_test)
df3 = pd.concat([df1, df2], axis=1, join = 'inner')
df3.to_csv("CSV_files/Testing_file.csv", index = False, encoding = 'utf-8')

# divide train data into train and cross validation 
x_train1, x_cv,  y_train1, y_cv = train_test_split(x_train,y_train, test_size = 0.20,random_state=0)
print("x_train_data : ", x_train1.shape, " x_crossV_data : ", x_cv.shape)

x_train :  (16, 49)  x_test :  (4, 49)
x_train_data :  (12, 49)  x_crossV_data :  (4, 49)


In [55]:
class Naive_Bayes():
    
    def create_module(self,x_train,y_train):
        # Fitting Naive Bayes to the Training set
        classifier = GaussianNB()
        classifier.fit(x_train, y_train)
        return classifier
        
def main():
    #class obj created
    obj  = Naive_Bayes()
    
    #split dataset into train, test and cross validation also save csv files
#     x_train, x_cv,  y_train, y_cv = obj_util.splitdata(x, y, "CSV_files/Naive_Bayes_test.csv", 0.20,0.20)

#     # Feature Scalling for train and test data 
#     x_train = obj_util.feature_Scaling(x_train)
#     x_test = obj_util.feature_Scaling(x_test)
    
#     sc = StandardScaler()
#     x_train = sc.fit_transform(x_train)
#     x_cv = sc.transform(x_cv)

    classifier = obj.create_module(x_train,y_train)
#     cl = nltk.NaiveBayesClassifier.train(x_train)
#     print(cl.show_most_informative_features(10))
#     print("\nModule created")
#     print("classifier object",classifier)

    y_pre = obj_util.y_prediction(x_cv, classifier)
#     print("\n\n y_prediction:",y_pre)
#     print(y_pre.shape)
    
    cm = obj_util.confusion_matrix(y_cv,y_pre)
    print("\n\nConfusion Matrix:\n",cm)
    
#     accuracy_score,average_precision,auc, f1_score_acc = obj_util.accuracy(y_pre,y_cv)
#     print('\n\nAverage accuracy_score:' , accuracy_score)
#     print('Average precision-recall score: {0:0.2f}'.format(average_precision))
#     print('Average Roc-AUC: %.3f' % auc)
#     print('Accuracy by F1-score:',f1_score_acc)

#     obj_util.create_piklefile(classifier, sc, "Naive_Bayes.pkl")
    print("\nModel created with pickle file")
    
    print("-------------------------CROSS Validation-------------------------------------")
     # predict y
    y_pre = obj_util.y_prediction(x_cv, classifier)
#     print(y_pre, y_cv)
    # Calculate accuracy
#     accuracy = average_precision_score(y_cv, y_pre)* 100
#     print("\n Accuracy: average_precision_score :", accuracy)

#     print("\n",classification_report(y_cv, y_pre))

#     y_pred_prob = classifier.predict_proba(x_cv)[:,1]

#     print("\n ROC curve \n")
#     fpr, tpr, thresholds =  metrics.roc_curve(y_cv, y_pred_prob)
#     # create plot
#     plt.plot(fpr, tpr, label='ROC curve')
#     plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
#     plt.title('ROC Curve')
#     plt.xlim([-0.02, 1])
#     plt.ylim([0, 1.02])
#     plt.legend(loc="lower right")

if __name__ == '__main__':
    main()




Confusion Matrix:
 [[4]]

Model created with pickle file
-------------------------CROSS Validation-------------------------------------
